<a href="https://colab.research.google.com/github/siddtheshah/vc_modeling/blob/master/category_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from copy import deepcopy

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
file_names = os.listdir("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/")
df_names = [x[:-4] for x in file_names]
print(df_names)

dfs = [pd.read_csv("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/"+x) for x in file_names]
df_dict = dict(zip(df_names, dfs))
print(df_dict.keys())

['category_groups', 'funding_rounds', 'people', 'checksum', 'people_descriptions', 'investors', 'organization_descriptions', 'investment_partners', 'event_appearances', 'organizations', 'org_parents', 'jobs', 'acquisitions', 'funds', 'ipos', 'degrees', 'investments', 'events']
dict_keys(['category_groups', 'funding_rounds', 'people', 'checksum', 'people_descriptions', 'investors', 'organization_descriptions', 'investment_partners', 'event_appearances', 'organizations', 'org_parents', 'jobs', 'acquisitions', 'funds', 'ipos', 'degrees', 'investments', 'events'])


In [4]:
import collections

org_info = df_dict['organizations']

# Filter down the data purely to investees
org_info = org_info[org_info['primary_role'] == 'company']

# Select organization data
# Clean the df

comma_sep_list_vars = ['category_list', 'category_groups_list']
categorical_vars = ['region']
other_vars = ['uuid']
org_info_selected = org_info[other_vars + categorical_vars + comma_sep_list_vars]
org_info_selected = org_info_selected.dropna()
org_info_selected = org_info_selected.reset_index(drop=True)

vocabulary = collections.Counter()

iter = 0
test_stop = -1
total_rows = np.shape(org_info_selected)[0]
print("Processing ", total_rows, " rows.")
for (i, row) in org_info_selected.iterrows():
  if iter == test_stop:
    break
  iter += 1
  if iter % 100000 == 0:
    print(iter, " of ", total_rows)

  for col, _ in row.iteritems():
    if col in comma_sep_list_vars:
      # Break categories into unigrams
      token_lists = [x.split(' ') for x in row[col].strip().split(',')]
      tokens = []
      for token_list in token_lists:
        for token in token_list:
          tokens.append(token)

      for token in tokens:
        vocabulary[token] += 1

# Filter low count words
low_count_threshold = 15
keys_to_remove = []
for word, count in vocabulary.items():
  if count < low_count_threshold:
    keys_to_remove.append(word)

for key in keys_to_remove:
  del vocabulary[key]

# Now rank and filter by document frequency. The top half frequent words will be dropped
key_list = list(vocabulary.keys())
fraction_to_drop = .25
chop = sorted(key_list, key=lambda x: vocabulary[x])[-int(.5*len(key_list)):]
for key in chop:
  del vocabulary[key]

# Create index to word mapping
word_to_index = {}
word_index = 0
for word, count in vocabulary.items():
  word_to_index[word] = word_index
  word_index += 1

print(word_to_index)
vocabulary_size = word_index

Processing  808944  rows.
100000  of  808944
200000  of  808944
300000  of  808944
400000  of  808944
500000  of  808944
600000  of  808944
700000  of  808944
800000  of  808944
{'SMS': 0, 'EBooks': 1, 'Embedded': 2, 'Hedge': 3, 'Funds': 4, 'Contact': 5, 'Knowledge': 6, 'Currency': 7, 'GPS': 8, 'Bookmarking': 9, 'Browser': 10, 'Extensions': 11, 'Dating': 12, 'Chat': 13, 'Forums': 14, 'Celebrity': 15, 'Radio': 16, 'Facebook': 17, 'World': 18, 'Podcast': 19, 'Stock': 20, 'Exchange': 21, 'Discovery': 22, 'Ediscovery': 23, 'Physical': 24, 'Flash': 25, 'Concerts': 26, 'Nightlife': 27, 'National': 28, 'Freelance': 29, 'to': 30, 'Classifieds': 31, 'Reviews': 32, 'Cooking': 33, 'Recipes': 34, 'Teenagers': 35, 'WebOS': 36, 'Promotion': 37, 'Universities': 38, 'Virtualization': 39, 'on': 40, 'Demand': 41, 'Angel': 42, 'Transaction': 43, 'Outdoors': 44, 'Recreation': 45, 'MMO': 46, 'Tobacco': 47, 'Alumni': 48, 'Reputation': 49, 'Image': 50, 'CMS': 51, 'Grocery': 52, 'IaaS': 53, 'Meeting': 54, 'In

In [5]:
from scipy.sparse import csr_matrix
import collections

# Load our filtered vocabulary if we've saved it on disk.

row_indices = []
col_indices = []
data_values = []

iter = 0
test_stop = -1
total_rows = np.shape(org_info_selected)[0]
print("Processing ", total_rows, " rows.")
for (i, row) in org_info_selected.iterrows():
  if iter == test_stop:
    break
  iter += 1
  if iter % 100000 == 0:
    print(iter, " of ", total_rows)

  for col, _ in row.iteritems():
    if col in comma_sep_list_vars:
      # Break categories into unigrams
      unigram_counts = collections.Counter()
      token_lists = [x.split(' ') for x in row[col].strip().split(',')]
      for token_list in token_lists:
        for token in token_list:
          unigram_counts[token] += 1
      
      for unigram, count in unigram_counts.items():
        if unigram in vocabulary:
          row_indices.append(i)
          col_indices.append(word_to_index[unigram])
          data_values.append(count)
    elif col in categorical_vars:
      value = row[col]
      if value in vocabulary:
        row_indices.append(i)
        col_indices.append(word_to_index[value])
        data_values.append(1)

unclustered = csr_matrix((data_values, (row_indices, col_indices)))
print(unclustered.toarray().shape)

Processing  808944  rows.
100000  of  808944
200000  of  808944
300000  of  808944
400000  of  808944
500000  of  808944
600000  of  808944
700000  of  808944
800000  of  808944
(808944, 375)


In [8]:
import scipy.sparse

scipy.sparse.save_npz('/category_features.npz', unclustered)
companies = org_info_selected['uuid']
companies.to_csv('/companies.csv')